In [1]:
# importar paquetes a ocupar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os

In [2]:
# importar archivos csv# Abrir el archivo comprimido
with zipfile.ZipFile('datasets/teams/battlelog_teams.zip', 'r') as zip_ref:
    # Extraer el archivo CSV
    zip_ref.extractall('datasets/teams/')

crossBattlelog = pd.read_csv('datasets/teams/battlelog_teams.csv', index_col=0)

os.remove('datasets/teams/battlelog_teams.csv')

crossBattlelog.info()

C:\Users\alniquia\AppData\Local\Temp\ipykernel_14780\2723443249.py:6: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  crossBattlelog = pd.read_csv('datasets/teams/battlelog_teams.csv', index_col=0)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 418836 entries, 0 to 418835
Data columns (total 63 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   battleTime                             418836 non-null  object 
 1   playertag                              418836 non-null  object 
 2   event.id                               418836 non-null  int64  
 3   event.mode                             418836 non-null  object 
 4   event.map                              418836 non-null  object 
 5   battle.mode                            418836 non-null  object 
 6   battle.type                            418836 non-null  object 
 7   battle.result                          418836 non-null  object 
 8   battle.duration                        418836 non-null  float64
 9   battle.trophyChange                    309471 non-null  float64
 10  battle.starPlayer.tag                  350888 non-null  

In [3]:
crossBattlelog['event.map'] = crossBattlelog['event.map'].replace('Belle`s Rock',"Belle's Rock")
crossBattlelog['event.map'].value_counts()

Super Beach         41516
Pinhole Punt        35015
Sneaky Fields       28870
Center Stage        21546
Out in the Open     16799
Beach Ball          16017
Hard Rock Mine      15520
Sunny Soccer        13605
Crystal Arcade      12154
Bridge Too Far      12133
Dueling Beetles     12007
Double Swoosh       11473
Safe Zone           10969
Backyard Bowl       10805
Shooting Star       10732
Ring of Fire        10723
Layer Cake          10001
Canal Grande         9708
Open Zone            9481
Belle's Rock         8262
Goldarm Gulch        8154
X Marks the Spot     8143
Pit Stop             8101
Deep Diner           7797
New Perspective      6259
Undermine            6221
Riverside            6005
Minecart Madness     5666
Dry Season           5615
Split                5225
Tornado Ring         4927
Snaked Assault       4760
Flaring Phoenix      3983
Gem Fort             3522
Controller Chaos     3365
Hot Potato           3326
Excel                3104
Kaboom Canyon        2309
Quarter Poun

In [4]:
# importar archivos csv
players = pd.read_csv('datasets/players/players.csv', index_col=0)
players.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86067 entries, 0 to 86066
Data columns (total 20 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   tag                                   86067 non-null  object 
 1   name                                  86066 non-null  object 
 2   nameColor                             83621 non-null  object 
 3   icon                                  5529 non-null   object 
 4   trophies                              86067 non-null  int64  
 5   highestTrophies                       86067 non-null  int64  
 6   highestPowerPlayPoints                62893 non-null  float64
 7   expLevel                              86067 non-null  int64  
 8   expPoints                             86067 non-null  int64  
 9   isQualifiedFromChampionshipChallenge  86067 non-null  bool   
 10  3vs3Victories                         86067 non-null  int64  
 11  soloVictories  

C:\Users\alniquia\AppData\Local\Temp\ipykernel_14780\611356329.py:2: DtypeWarning: Columns (4,16) have mixed types. Specify dtype option on import or set low_memory=False.
  players = pd.read_csv('datasets/players/players.csv', index_col=0)


In [5]:
# importar archivos csv
brawlers = pd.read_csv('datasets/brawlers/brawlers_stats.csv', index_col=0)
brawlers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 0 to 63
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              64 non-null     int64  
 1   name            64 non-null     object 
 2   starPowers      64 non-null     object 
 3   gadgets         64 non-null     object 
 4   Brawler         64 non-null     object 
 5   Health          64 non-null     float64
 6   Rarity          64 non-null     object 
 7   Class           64 non-null     object 
 8   Movement_Speed  64 non-null     object 
 9   Range           63 non-null     object 
 10  Reload          63 non-null     object 
 11  Speed_Num       64 non-null     int64  
 12  Range_Num       64 non-null     float64
 13  Reload_Num      63 non-null     float64
 14  Role            64 non-null     object 
dtypes: float64(3), int64(2), object(10)
memory usage: 8.0+ KB


In [6]:
# crear columna playerteam
for i in range(len(crossBattlelog)):
	if any(crossBattlelog.loc[i, 'playertag'] == tag for tag in [crossBattlelog.loc[i, 'battle.team1.player1.tag'], crossBattlelog.loc[i, 'battle.team1.player2.tag'], crossBattlelog.loc[i, 'battle.team1.player3.tag']]):
		crossBattlelog.loc[i, 'player_team'] = 'team1'
	elif any(crossBattlelog.loc[i, 'playertag'] == tag for tag in [crossBattlelog.loc[i, 'battle.team2.player1.tag'], crossBattlelog.loc[i, 'battle.team2.player2.tag'], crossBattlelog.loc[i, 'battle.team2.player3.tag']]):
		crossBattlelog.loc[i, 'player_team'] = 'team2'
	else:
		print("error in team assignation for id " + str(i))

crossBattlelog['player_team'].value_counts()

team1    210663
team2    208173
Name: player_team, dtype: int64

In [7]:
# crear columna winner_team
for i, result in enumerate(crossBattlelog['battle.result']):

	player_team = crossBattlelog.loc[i, 'player_team']
	
	if result == 'draw':
		crossBattlelog.loc[i, 'winner_team'] = 'draw'
	elif (player_team == 'team1' and result == 'victory') or (player_team == 'team2' and result == 'defeat'):
		crossBattlelog.loc[i, 'winner_team'] = 'team1'
	elif (player_team == 'team1' and result == 'defeat') or (player_team == 'team2' and result == 'victory'):
		crossBattlelog.loc[i, 'winner_team'] = 'team2'
	else:
		print("error in winner assignation for id " + str(i))

crossBattlelog['winner_team'].value_counts()

team1    210561
team2    205611
draw       2664
Name: winner_team, dtype: int64

In [8]:
# agregar columna identificando batallas low_trophy

def check_low_trophy(val, threshold=250):
	if val < threshold:
		return True
	else:
		return False

crossBattlelog['low_trophy'] = False

cols = ['battle.team1.player1.brawler.trophies', 'battle.team1.player2.brawler.trophies', 'battle.team1.player3.brawler.trophies', 'battle.team2.player1.brawler.trophies', 'battle.team2.player2.brawler.trophies', 'battle.team2.player3.brawler.trophies']

for item in(cols):
	for i, trophies in enumerate(crossBattlelog[item]):
		if crossBattlelog.loc[i, 'low_trophy'] == False:
			crossBattlelog.loc[i, 'low_trophy'] = check_low_trophy(trophies)
	
crossBattlelog = crossBattlelog[crossBattlelog['low_trophy'] == False].reset_index(drop=True)

crossBattlelog['low_trophy'].value_counts()

False    273248
Name: low_trophy, dtype: int64

In [9]:
# format date
# for i, battleTime in enumerate(crossBattlelog['battleTime']):
# 	crossBattlelog.loc[i, 'battleTime'] = pd.to_datetime(battleTime, format='%Y-%m-%d')

crossBattlelog['battleTime'] = pd.to_datetime(crossBattlelog['battleTime'], format='%Y-%m-%d').dt.date

crossBattlelog['battleTime'].value_counts()

2023-01-21    34516
2023-01-23    31468
2023-01-28    26295
2023-01-20    26042
2023-01-19    25191
2023-01-24    23131
2023-01-26    22524
2023-01-25    15129
2023-01-22    14352
2023-01-27     7670
2023-01-18     5889
2023-01-15     3855
2023-01-02     3758
2023-01-05     3182
2023-01-14     3045
2023-01-16     2926
2023-01-04     2629
2023-01-06     2543
2023-01-09     2517
2023-01-03     2436
2023-01-17     2209
2023-01-13     2158
2023-01-08     2069
2023-01-10     2054
2023-01-07     1605
2023-01-12     1293
2023-01-01     1087
2023-01-11      570
2022-12-31      325
2022-12-30      254
2022-12-29       99
2022-12-27       57
2022-12-13       43
2022-12-22       38
2022-12-26       34
2022-12-28       28
2022-12-23       25
2022-10-15       22
2022-12-25       22
2022-12-16       20
2022-12-19       17
2022-12-20       16
2022-12-24       12
2022-10-28       10
2022-12-21       10
2022-11-11        9
2022-10-26        9
2022-11-01        9
2022-10-30        9
2022-10-10        8


In [10]:
# crear columna winner
def clean_winner(team):

	dfw = crossBattlelog[crossBattlelog['player_team'] == team].reset_index(drop = True)

	for i, winner_team in enumerate(dfw['winner_team']):
		if winner_team == 'draw':
			winner = 'd'
		elif winner_team == team:
			winner = 'y'
		else:
			winner = 'n'
		
		dfw.loc[i, 'winner'] = winner

	dfw = dfw.drop('winner_team', axis=1)

	dfw = dfw.rename(columns={
		'battle.' + team + '.player1.tag': 'battle.player1.tag'
		,'battle.' + team + '.player1.brawler.name': 'battle.player1.brawler.name'
		,'battle.' + team + '.player1.brawler.power': 'battle.player1.brawler.power'
		,'battle.' + team + '.player1.brawler.trophies': 'battle.player1.brawler.trophies'
		,'battle.' + team + '.player2.tag': 'battle.player2.tag'
		,'battle.' + team + '.player2.brawler.name': 'battle.player2.brawler.name'
		,'battle.' + team + '.player2.brawler.power': 'battle.player2.brawler.power'
		,'battle.' + team + '.player2.brawler.trophies': 'battle.player2.brawler.trophies'
		,'battle.' + team + '.player3.tag': 'battle.player3.tag'
		,'battle.' + team + '.player3.brawler.name': 'battle.player3.brawler.name'
		,'battle.' + team + '.player3.brawler.power': 'battle.player3.brawler.power'
		,'battle.' + team + '.player3.brawler.trophies': 'battle.player3.brawler.trophies'
	})

	dfw = dfw[[	
		'battleTime'
		,'event.mode'
		,'event.map'
		,'battle.type'
		,'battle.duration'
		,'battle.trophyChange'
		,'battle.starPlayer.tag'
 		,'battle.starPlayer.brawler.name'
 		,'battle.starPlayer.brawler.power'
 		,'battle.starPlayer.brawler.trophies'
		,'battle.player1.tag'
		,'battle.player1.brawler.name'
		,'battle.player1.brawler.power'
		,'battle.player1.brawler.trophies'
		,'battle.player2.tag'
		,'battle.player2.brawler.name'
		,'battle.player2.brawler.power'
		,'battle.player2.brawler.trophies'
		,'battle.player3.tag'
		,'battle.player3.brawler.name'
		,'battle.player3.brawler.power'
		,'battle.player3.brawler.trophies'
		,'winner'
	]]

	if team == 'team1':
		oteam = 'team2'
	else:
		oteam = 'team1'
	
	dfl = crossBattlelog[crossBattlelog['player_team'] == team].reset_index(drop = True)

	for i, winner_team in enumerate(dfl['winner_team']):
		if winner_team == 'draw':
			winner = 'd'
		elif winner_team == oteam:
			winner = 'y'
		else:
			winner = 'n'
		
		dfl.loc[i, 'winner'] = winner

	dfl = dfl.drop('winner_team', axis=1)

	dfl = dfl.rename(columns={
		'battle.' + oteam + '.player1.tag': 'battle.player1.tag'
		,'battle.' + oteam + '.player1.brawler.name': 'battle.player1.brawler.name'
		,'battle.' + oteam + '.player1.brawler.power': 'battle.player1.brawler.power'
		,'battle.' + oteam + '.player1.brawler.trophies': 'battle.player1.brawler.trophies'
		,'battle.' + oteam + '.player2.tag': 'battle.player2.tag'
		,'battle.' + oteam + '.player2.brawler.name': 'battle.player2.brawler.name'
		,'battle.' + oteam + '.player2.brawler.power': 'battle.player2.brawler.power'
		,'battle.' + oteam + '.player2.brawler.trophies': 'battle.player2.brawler.trophies'
		,'battle.' + oteam + '.player3.tag': 'battle.player3.tag'
		,'battle.' + oteam + '.player3.brawler.name': 'battle.player3.brawler.name'
		,'battle.' + oteam + '.player3.brawler.power': 'battle.player3.brawler.power'
		,'battle.' + oteam + '.player3.brawler.trophies': 'battle.player3.brawler.trophies'
	})

	dfl = dfl[[	
		'battleTime'
		,'event.mode'
		,'event.map'
		,'battle.type'
		,'battle.duration'
		,'battle.trophyChange'
		,'battle.starPlayer.tag'
 		,'battle.starPlayer.brawler.name'
 		,'battle.starPlayer.brawler.power'
 		,'battle.starPlayer.brawler.trophies'
		,'battle.player1.tag'
		,'battle.player1.brawler.name'
		,'battle.player1.brawler.power'
		,'battle.player1.brawler.trophies'
		,'battle.player2.tag'
		,'battle.player2.brawler.name'
		,'battle.player2.brawler.power'
		,'battle.player2.brawler.trophies'
		,'battle.player3.tag'
		,'battle.player3.brawler.name'
		,'battle.player3.brawler.power'
		,'battle.player3.brawler.trophies'
		,'winner'
	]]

	return pd.concat([dfw, dfl], ignore_index=True)

team1 = clean_winner('team1')
team2 = clean_winner('team2')

battlelogProcess = pd.concat([team1, team2], ignore_index=True).reset_index(drop = True)
battlelogProcess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546496 entries, 0 to 546495
Data columns (total 23 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   battleTime                          546496 non-null  object 
 1   event.mode                          546496 non-null  object 
 2   event.map                           546496 non-null  object 
 3   battle.type                         546496 non-null  object 
 4   battle.duration                     546496 non-null  float64
 5   battle.trophyChange                 542194 non-null  float64
 6   battle.starPlayer.tag               546494 non-null  object 
 7   battle.starPlayer.brawler.name      546494 non-null  object 
 8   battle.starPlayer.brawler.power     546494 non-null  float64
 9   battle.starPlayer.brawler.trophies  546494 non-null  float64
 10  battle.player1.tag                  546496 non-null  object 
 11  battle.player1.brawler.nam

In [11]:
# agregar brawler stats
def add_brawlers(df, prefix):
	df = pd.merge(left=df, right=brawlers.add_prefix(prefix), on=prefix + 'name', how='left', suffixes=('','_b'))
	return df

battlelogProcess = add_brawlers(battlelogProcess, 'battle.player1.brawler.')
battlelogProcess = add_brawlers(battlelogProcess, 'battle.player2.brawler.')
battlelogProcess = add_brawlers(battlelogProcess, 'battle.player3.brawler.')

battlelogProcess.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 546496 entries, 0 to 546495
Data columns (total 65 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   battleTime                             546496 non-null  object 
 1   event.mode                             546496 non-null  object 
 2   event.map                              546496 non-null  object 
 3   battle.type                            546496 non-null  object 
 4   battle.duration                        546496 non-null  float64
 5   battle.trophyChange                    542194 non-null  float64
 6   battle.starPlayer.tag                  546494 non-null  object 
 7   battle.starPlayer.brawler.name         546494 non-null  object 
 8   battle.starPlayer.brawler.power        546494 non-null  float64
 9   battle.starPlayer.brawler.trophies     546494 non-null  float64
 10  battle.player1.tag                     546496 non-null  

In [91]:
# unificar bases de brawlers
bk1 = battlelogProcess.rename(columns={'battle.player1.brawler.name': 'brawlername'})
bk2 = battlelogProcess.rename(columns={'battle.player2.brawler.name': 'brawlername'})
bk3 = battlelogProcess.rename(columns={'battle.player3.brawler.name': 'brawlername'})

brawler = pd.concat([
bk1[['battleTime','event.mode','event.map','battle.type','brawlername','winner']], 
bk2[['battleTime','event.mode','event.map','battle.type','brawlername','winner']], 
bk3[['battleTime','event.mode','event.map','battle.type','brawlername','winner']]
], ignore_index=True)

brawler

,battleTime,event.mode,event.map,battle.type,brawlername,winner
0,2023-01-28,brawlBall,Super Beach,ranked,NITA,y
1,2023-01-28,brawlBall,Super Beach,ranked,NITA,n
2,2023-01-28,brawlBall,Super Beach,ranked,NITA,y
3,2023-01-28,brawlBall,Super Beach,ranked,JANET,y
4,2023-01-28,brawlBall,Super Beach,ranked,JANET,y
...,...,...,...,...,...,...
1639483,2022-12-31,heist,Bridge Too Far,ranked,AMBER,n
1639484,2023-01-02,hotZone,Quarter Pounder,ranked,BULL,n
1639485,2023-01-02,gemGrab,Double Swoosh,ranked,ASH,y
1639486,2023-01-01,hotZone,Quarter Pounder,ranked,BEA,n


In [92]:
brawler['winner'].value_counts()

y    813894
n    813894
d     11700
Name: winner, dtype: int64

In [112]:
# crear pivot tables
pivot_brawler = brawler.pivot_table(index=['battleTime','brawlername','event.mode','event.map'], columns='winner', values='winner', aggfunc='size', fill_value=0).reset_index(drop=False)
pivot_brawler

winner,battleTime,brawlername,event.mode,event.map,d,n,y
0,2022-10-10,BELLE,knockout,Belle's Rock,0,4,1
1,2022-10-10,BONNIE,knockout,Belle's Rock,0,0,3
2,2022-10-10,BROCK,knockout,Belle's Rock,0,3,2
3,2022-10-10,BUZZ,knockout,Belle's Rock,0,0,1
4,2022-10-10,BYRON,knockout,Belle's Rock,0,1,0
...,...,...,...,...,...,...,...
14876,2023-01-28,TICK,gemGrab,Deep Diner,0,95,58
14877,2023-01-28,TICK,gemGrab,Hard Rock Mine,0,146,68
14878,2023-01-28,TICK,heist,Bridge Too Far,0,30,11
14879,2023-01-28,TICK,hotZone,Controller Chaos,0,211,46


In [113]:
# agregar columna totales, win y draw

pivot_brawler['total_games'] = pivot_brawler['d'] + pivot_brawler['n'] + pivot_brawler['y']
# pivot_brawler['winrate'] = pivot_brawler['y'] / pivot_brawler['total_games']

# total_games_event = pivot_brawler[['battleTime','event.mode','event.map','total_games']].groupby(['battleTime','event.mode','event.map']).sum()

# pivot_brawler = pivot_brawler.merge(total_games_event, on=['battleTime','event.mode','event.map'], suffixes=('', '_event'))

# pivot_brawler['userate'] = pivot_brawler['total_games'] / pivot_brawler['total_games_event']

pivot_brawler = pivot_brawler.sort_values(by=['brawlername','battleTime','event.mode','event.map'], ascending=False).reset_index(drop=True)

pivot_brawler

winner,battleTime,brawlername,event.mode,event.map,d,n,y,total_games
0,2023-01-28,TICK,knockout,Flaring Phoenix,2,318,232,552
1,2023-01-28,TICK,hotZone,Controller Chaos,0,211,46,257
2,2023-01-28,TICK,heist,Bridge Too Far,0,30,11,41
3,2023-01-28,TICK,gemGrab,Hard Rock Mine,0,146,68,214
4,2023-01-28,TICK,gemGrab,Deep Diner,0,95,58,153
...,...,...,...,...,...,...,...,...
14876,2022-12-27,8-BIT,gemGrab,Minecart Madness,0,1,0,1
14877,2022-12-26,8-BIT,gemGrab,Minecart Madness,0,2,0,2
14878,2022-10-30,8-BIT,heist,Safe Zone,0,1,0,1
14879,2022-10-15,8-BIT,gemGrab,Hard Rock Mine,0,0,2,2


In [98]:
# export brawler_by_event_map
pivot_brawler.to_csv('datasets/teams/brawler_by_event_map.csv')

In [233]:
# definición de funciones
# función crea cuadrantes entre dos columnas

def quadrants(df, col_a, col_b):
	result = np.where(((df[col_a] / df[col_a].quantile(0.5)) > 1) & ((df[col_b] / df[col_b].quantile(0.5)) > 1), 'Q1'
			,np.where(((df[col_a] / df[col_a].quantile(0.5)) < 1) & ((df[col_b] / df[col_b].quantile(0.5)) > 1), 'Q2'
			,np.where(((df[col_a] / df[col_a].quantile(0.5)) < 1) & ((df[col_b] / df[col_b].quantile(0.5)) < 1), 'Q3'
			,'Q4')))
	return result

# función que crea distribución overall
def dist_overall():
	df = pivot_brawler.groupby(['brawlername']).sum(numeric_only=True).reset_index()
	df['winrate'] = df['y'] / df['total_games']
	df['userate'] = df['total_games'] / sum(df['total_games'])
	df['percentile_quadrant'] = quadrants(df, 'winrate', 'userate')
	return df

# función que crea distribución por brawler general
def dist_gamemode(gamemode):
	df = pivot_brawler[(pivot_brawler['event.mode']==gamemode)].groupby(['brawlername','event.mode']).sum(numeric_only=True).reset_index()
	df['winrate'] = df['y'] / df['total_games']
	df['userate'] = df['total_games'] / sum(df['total_games'])
	df['percentile_quadrant'] = quadrants(df, 'winrate', 'userate')
	return df

# función que crea distribución por brawler
def dist_brawler(brawler):
	df = pivot_brawler[(pivot_brawler['brawlername']==brawler)].groupby(['event.mode']).sum(numeric_only=True).reset_index()
	df['winrate'] = df['y'] / df['total_games']
	df['userate'] = df['total_games'] / sum(df['total_games'])
	return df

# función que crea distribución por gamemode y map
def dist_gamemode_map(gamemode, map):
	df = pivot_brawler[(pivot_brawler['event.mode']==gamemode) & (pivot_brawler['event.map']==map)].groupby(['brawlername']).sum(numeric_only=True).reset_index()
	df['winrate'] = df['y'] / df['total_games']
	df['userate'] = df['total_games'] / sum(df['total_games'])
	df['percentile_quadrant'] = quadrants(df, 'winrate', 'userate')
	return df

# función que crea distribución por fecha
def dist_brawler_date(brawler):
	df_gm = pivot_brawler.groupby(['battleTime']).sum(numeric_only=True).reset_index()
	df = pivot_brawler[pivot_brawler['brawlername'] == brawler].groupby(['battleTime','brawlername']).sum(numeric_only=True).reset_index()
	df = df[df['battleTime'] >= pd.Timestamp('2023-01-19').date()]
	df['winrate'] = df['y'] / df['total_games']

	df = pd.merge(df, df_gm[['battleTime','total_games']], on='battleTime', how='left', suffixes=('', '_day'))

	df['userate'] = df['total_games'] / df['total_games_day']
	return df

In [224]:
# definir estilo de plot
print(plt.style.available)
plt.style.use('seaborn-v0_8-dark-palette')

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [225]:
#estilo sns
print(sns.axes_style())
sns.set_theme('paper')

{'axes.facecolor': '#EAEAF2', 'axes.edgecolor': 'white', 'axes.grid': True, 'axes.axisbelow': True, 'axes.labelcolor': '.15', 'figure.facecolor': 'white', 'grid.color': 'white', 'grid.linestyle': '-', 'text.color': '.15', 'xtick.color': '.15', 'ytick.color': '.15', 'xtick.direction': 'out', 'ytick.direction': 'out', 'lines.solid_capstyle': <CapStyle.round: 'round'>, 'patch.edgecolor': 'w', 'patch.force_edgecolor': True, 'image.cmap': 'rocket', 'font.family': ['sans-serif'], 'font.sans-serif': ['Arial', 'DejaVu Sans', 'Liberation Sans', 'Bitstream Vera Sans', 'sans-serif'], 'xtick.bottom': False, 'xtick.top': False, 'ytick.left': False, 'ytick.right': False, 'axes.spines.left': True, 'axes.spines.bottom': True, 'axes.spines.right': True, 'axes.spines.top': True}


In [226]:
# winrate / userate overall
data = dist_overall()

fig, ax = plt.subplots(figsize=(8,8))

sns.scatterplot(
	data=data
	,x="winrate"
	,y="userate"
	,hue='percentile_quadrant'
	,ax=ax
)

for i, point in data.iterrows():
		ax.text(point.winrate + 0.001, point.userate + 0.001, point.brawlername, fontsize=6.5)

yticks = ax.get_yticks()
xticks = ax.get_xticks()

ax.set_yticks(yticks)
ax.set_xticks(xticks)

ax.set_yticklabels(["{:,.00%}".format(y) for y in yticks])
ax.set_xticklabels(["{:,.00%}".format(x) for x in xticks])

plt.title('Overall Winrate / Userate')

plt.savefig('plots/overall/overall.png', dpi=300, bbox_inches='tight')

plt.close()

# data[data['percentile_quadrant']=='Q1'].sort_values(by='winrate', ascending=False).to_csv('datasets/best_brawlers/overall/overall_q1.csv', index=False)

In [227]:
# winrate / userate
gamemodes = pivot_brawler['event.mode'].drop_duplicates().tolist()

for i, gm in enumerate(gamemodes):
    data = dist_gamemode(gm)

    fig, ax = plt.subplots(figsize=(8,7))

    sns.scatterplot(
        data=data
        ,x="winrate"
        ,y="userate"
        ,hue='percentile_quadrant'
        ,ax=ax
    )

    for i, point in data.iterrows():
        ax.text(point.winrate + 0.001, point.userate + 0.001, point.brawlername, fontsize=6.5)

    yticks = ax.get_yticks()
    xticks = ax.get_xticks()

    ax.set_yticks(yticks)
    ax.set_xticks(xticks)

    ax.set_yticklabels(["{:,.00%}".format(y) for y in yticks])
    ax.set_xticklabels(["{:,.00%}".format(x) for x in xticks])
    
    plt.title(str(data['event.mode'][0]) + '\n' + 'total games: ' + str(sum(data['total_games'])))

    plt.savefig('plots/gamemode/' + gm + '.png', dpi=300, bbox_inches='tight')
    
    plt.close()

    # data[data['percentile_quadrant']=='Q1'].sort_values(by='winrate', ascending=False).to_csv('datasets/best_brawlers/gamemode/' + gm + '.csv', index=False)

In [228]:
# winrate / userate
gamemodes = pivot_brawler['event.mode'].drop_duplicates().tolist()

for j, gm in enumerate(gamemodes):

	maps = pivot_brawler['event.map'][pivot_brawler['event.mode'] == gm].drop_duplicates().tolist()

	for i, map in enumerate(maps):
		data = dist_gamemode_map(gm, map)

		fig, ax = plt.subplots(figsize=(8,7))

		sns.scatterplot(
			data=data
			,x="winrate"
			,y="userate"
        	,hue='percentile_quadrant'
			,ax=ax
		)

		for i, point in data.iterrows():
			ax.text(point.winrate + 0.001, point.userate + 0.001, point.brawlername + '\n g:' + str(point.total_games) + ' w:' + str(point.y), fontsize=6.5)

		yticks = ax.get_yticks()
		xticks = ax.get_xticks()

		ax.set_yticks(yticks)
		ax.set_xticks(xticks)

		ax.set_yticklabels(["{:,.00%}".format(y) for y in yticks])
		ax.set_xticklabels(["{:,.00%}".format(x) for x in xticks])

		plt.title(gm + '\n' + map + '\n' + 'total games: ' + str(sum(data['total_games'])))

		plt.savefig('plots/gamemode_map/' + gm + '_' + map + '.png', dpi=300, bbox_inches='tight')

		plt.close()

		# data[data['percentile_quadrant']=='Q1'].sort_values(by='winrate', ascending=False).to_csv('datasets/best_brawlers/gamemode_map/' + gm + '_' + map + '.csv', index=False)

In [229]:
# winrate / userate by brawler
brawlers = pivot_brawler['brawlername'].drop_duplicates().tolist()

for j, br in enumerate(brawlers):

	data = dist_brawler(br)

	fig, ax = plt.subplots(figsize=(5,5))

	sns.scatterplot(
		data=data
		,x="winrate"
		,y="userate"
		,ax=ax
	)

	for i, point in data.iterrows():
		ax.text(point.winrate + 0.0002, point.userate + 0.0002, point[0] + '\n g:' + str(point.total_games) + ' w:' + str(point.y), fontsize=6.5)

	yticks = ax.get_yticks()
	xticks = ax.get_xticks()

	ax.set_yticks(yticks)
	ax.set_xticks(xticks)

	ax.set_yticklabels(["{:,.00%}".format(y) for y in yticks])
	ax.set_xticklabels(["{:,.00%}".format(x) for x in xticks])

	plt.title(br + '\n' + 'total games: ' + str(sum(data['total_games'])))

	plt.savefig('plots/brawler/' + br + '.png', dpi=300, bbox_inches='tight')

	plt.close()

	# data.sort_values(by='winrate', ascending=False).to_csv('datasets/best_brawlers/brawlers/' + br + '.csv', index=False)

In [234]:
# winrate / userate by date
brawlers = pivot_brawler['brawlername'].drop_duplicates().tolist()

for brawler in brawlers:

	data = dist_brawler_date(brawler)

	fig, ax = plt.subplots(2, 1, figsize=(8,8))

	ax[0].plot(data['battleTime'], data['winrate'], color='green', alpha=0.5, label='winrate', marker='o')

	plt.title('Overall Winrate / Userate' + '\n' + brawler)

	ax[1].bar(data['battleTime'], data['userate'], color='blue', alpha=0.5, label='userate')

	plt.savefig('plots/date/' + brawler + '.png', dpi=300, bbox_inches='tight')

	plt.close()

	# data[data['percentile_quadrant']=='Q1'].sort_values(by='winrate', ascending=False).to_csv('datasets/best_brawlers/overall/overall_q1.csv', index=False)